In [1]:
pip install pandas numpy opencv-python mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 3.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 4.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 MB 1.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.6/219.6 kB 3.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import time

In [6]:
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
)

I0000 00:00:1772027845.177230   19340 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1772027845.198020   19950 gl_context.cc:369] GL version: 3.0 (OpenGL ES 3.0 Mesa 25.0.7-0ubuntu0.24.04.2), renderer: Mesa Intel(R) HD Graphics 4000 (IVB GT2)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1772027845.272575   19947 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1772027845.308520   19946 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [7]:
def extract_hand(hand_landmarks):
    feat = []
    for lm in hand_landmarks.landmark:
        feat.extend([lm.x, lm.y, lm.z])
    return feat

In [ ]:
cap = cv2.VideoCapture(0)

current_gesture = "none"
collecting = False
records = []

print("""
q : quit
c : toggle collect
1-9 : set gesture (gesture_1, gesture_2...)
""")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    res = hands.process(rgb)

    L_feat = [0.0] * 63
    R_feat = [0.0] * 63
    L_exist = 0
    R_exist = 0

    if res.multi_hand_landmarks:
        for hand_lm, handedness in zip(
            res.multi_hand_landmarks,
            res.multi_handedness
        ):
            label = handedness.classification[0].label
            feat = extract_hand(hand_lm)

            if label == "Left":
                L_feat = feat
                L_exist = 1
            else:
                R_feat = feat
                R_exist = 1

            mp_draw.draw_landmarks(
                frame, hand_lm, mp_hands.HAND_CONNECTIONS
            )

    if collecting:
        row = (
            [current_gesture, L_exist, R_exist]
            + L_feat
            + R_feat
        )
        records.append(row)

    cv2.putText(
        frame,
        f"Gesture: {current_gesture} | Collecting: {collecting}",
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.7,
        (0, 255, 0) if collecting else (0, 0, 255),
        2
    )

    cv2.imshow("Landmark Collector", frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break
    elif key == ord("c"):
        collecting = not collecting
        print("Collecting:", collecting)
    elif ord("1") <= key <= ord("9"):
        current_gesture = f"gesture_{key - ord('0')}"
        print("Gesture set:", current_gesture)

cap.release()
cv2.destroyAllWindows()


q : quit
c : toggle collect
1-9 : set gesture (gesture_1, gesture_2...)



W0000 00:00:1772027855.466982   19946 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


KeyboardInterrupt: 

: 

In [8]:
columns = (
    ["gesture", "L_exist", "R_exist"]
    + [f"L_{a}{i}" for i in range(21) for a in ["x","y","z"]]
    + [f"R_{a}{i}" for i in range(21) for a in ["x","y","z"]]
)

df = pd.DataFrame(records, columns=columns)
df.to_csv("hand_landmarks.csv", index=False)
print("Saved:", df.shape)

Saved: (136, 129)
